# 📊 Análisis EDA - training_dataset.csv

## Prerequisito: Valido esquema antes de analizar

In [ ]:
import pandas as pd
import os
from validators.validation_schema import validate_training_dataset
import dotenv
dotenv.load_dotenv()

TRANING_DATA_PATH = os.getenv("TRAINING_DATA_PATH")

if not TRANING_DATA_PATH:
    raise ValueError("La variable de entorno 'TRAINING_DATA_PATH' no está definida.")

if not os.path.exists(TRANING_DATA_PATH):
    raise FileNotFoundError(f"La ruta especificada '{TRANING_DATA_PATH}' no existe.")

# Cargar dataset
df = pd.read_csv(TRANING_DATA_PATH)

# Validar estructura y contenido
df = validate_training_dataset(df)
print("✅ Dataset validado correctamente con pandera.")


## Seleccionar partidas de un determinado jugador

In [ ]:
import json
from pgn_utils import load_all_games_from_dir, get_game_hash
from tactical_analysis import detect_tactics_from_game

for game in load_all_games_from_dir(folder):
    game_hash = get_game_hash(game)
    if game_hash in analyzed:
        continue

    print(f"🔍 Analizando táctica: {game.headers.get('White', '?')} vs {game.headers.get('Black', '?')}")

    try:
        tags = detect_tactics_from_game(game)
        if tags:
            with open(f"{output_folder}/{game_hash}.json", "w", encoding="utf-8") as f:
                json.dump(tags, f, ensure_ascii=False, indent=2)

        # ✅ Guardamos sólo si no hubo excepción
        save_analyzed_hash(game_hash)

    except Exception as e:
        print(f"❌ Error al analizar partida {game_hash} - {game.headers.get('White', '?')} vs {game.headers.get('Black', '?')}: {e}")

In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar dataset
df = pd.read_csv('training_dataset.csv')
df.head()


## 1. Distribución de etiquetas de error

In [ ]:

df['error_label'].value_counts().plot(kind='bar', color='skyblue')
plt.title("Distribución de etiquetas de error")
plt.xlabel("Etiqueta")
plt.ylabel("Cantidad")
plt.grid(True)
plt.show()


## 2. Correlaciones entre variables estratégicas y errores

In [ ]:

variables = ['branching_factor', 'self_mobility', 'opponent_mobility', 'material_total']
correlation_matrix = df[variables + ['error_label']].corr(numeric_only=True)
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.title("Matriz de correlación")
plt.show()


## 3. Análisis por control del centro vs errores

In [ ]:

pd.crosstab(df['is_center_controlled'], df['error_label']).plot(kind='bar', stacked=True)
plt.title("Errores según control del centro")
plt.xlabel("¿Controla el centro?")
plt.ylabel("Cantidad")
plt.grid(True)
plt.show()


## 4. Fase de juego vs tipo de error

In [ ]:

pd.crosstab(df['phase'], df['error_label']).plot(kind='bar', stacked=True)
plt.title("Errores por fase de juego")
plt.xlabel("Fase")
plt.ylabel("Cantidad")
plt.grid(True)
plt.show()


## 5. Boxplot de movilidad vs error

In [ ]:

sns.boxplot(data=df, x='error_label', y='self_mobility')
plt.title("Movilidad propia según tipo de error")
plt.grid(True)
plt.show()


## 6. Error promedio en baja movilidad

In [ ]:
sns.barplot(data=df, x="is_low_mobility", y="score_diff", hue="classification")
plt.title("score_diff en posiciones con baja movilidad")
plt.tight_layout()
plt.show()

## 7. Correlaciones numéricas

In [ ]:
corr = df.corr(numeric_only=True)
sns.heatmap(corr, annot=True, cmap="coolwarm")
plt.title("Mapa de calor de correlaciones")
plt.tight_layout()
plt.show()

## 8. Conclusiones preliminares

- Las jugadas con baja movilidad muestran mayor score_diff promedio (peores resultados).
- Existen correlaciones entre movilidad propia, balance de material y score_diff.
- La distribución de errores es más frecuente en fases medias y finales con menos opciones legales.